In [95]:
from __future__ import division
import nltk
import re
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import brown
from nltk import bigrams, ngrams, trigrams 
from nltk.probability import FreqDist as FreqDist  
from numpy.polynomial.polynomial import polyfit
from collections import Counter
from string import digits
from scipy import stats
import operator
from collections import OrderedDict

In [96]:
from nltk.corpus import treebank,brown

corpus = brown.tagged_sents(tagset='universal')[:-100] 

tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0]
        tag= elem[1]

        if w not in word_dict:
            word_dict[w]=0

        if tag not in tag_dict:
            tag_dict[tag]=0

        word_dict[w]+=1
        tag_dict[tag]+=1

test_data= brown.tagged_sents(tagset='universal')[-100:]


In [97]:
corpus = brown.tagged_sents(tagset='universal')[:-100] 
corpus[0]

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

## Build the start, emission and transition probability matrices for the HMM

In [98]:
start={}
transition={}
emission={}
context={}
for line in corpus:
    previous='<s>'
    for word,tag in line:
        if(previous=='<s>'):                           # First tag of the sentence
            if tag not in context:
                context[tag]=0
            context[tag]+=1
            if tag not in start:
                start[tag]=0.0
            start[tag]+=1
            previous=tag
        else:                                          # Rest of the word-tag pairs
            if tag not in context:
                context[tag]=0
            context[tag]+=1
            if previous not in transition:
                transition[previous]={}
            if tag not in transition[previous]:
                transition[previous][tag]=0
            transition[previous][tag]+=1
            previous=tag
        if tag not in emission:
            emission[tag]={}
        if word not in emission[tag]:
            emission[tag][word]=0
        emission[tag][word]+=1
    if '</s>' not in transition[previous]:
        transition[previous]['</s>']=0
    transition[previous]['</s>']+=1

for tag in transition:                                 # Normalizing the counts into probability scores
    total_count=float(sum(transition[tag].values())) 
    for next_tag in transition[tag]:
        transition[tag][next_tag]/=(total_count)
for tag in start:                                      # Normalizing the counts into probability scores
    start[tag]/=float(len(corpus))

for tag in emission:                                   # Normalizing the counts into probability scores
    total_count=float(sum(emission[tag].values())) 
    for word in emission[tag]:
        emission[tag][word]/=(total_count)        

In [99]:
sentence='Hello how do you do'


def pos_tagger(sentence):
    viterbi_scores=dict()
    backpointer=dict()
    N=len(context.keys())   # Number of states except accept state '</s>'
    #T=len(sentence.split())
    T=len(sentence)
    words=['dummy']
    #words.extend(sentence.split())
    words.extend(sentence)
    for s in context.keys():                                      # Initialization
        viterbi_scores[s,1]=start.get(s,0)*emission[s].get(words[1],0)
        backpointer[s,1]=0
    for t in range(2,T+1):                                        # Recursion
        for s in context.keys():
            max_prob=-1
            for s1 in context.keys():
                prob=viterbi_scores[s1,t-1]
                prob*=transition[s1].get(s,0)
                prob*=emission[s].get(words[t],0)
                if(prob>max_prob):
                    viterbi_scores[s,t]=prob
                    backpointer[s,t]=s1
                    max_prob=prob
            #print(t,max_prob,backpointer[s,t])
    max_prob=-1.0
    for s in context.keys():                                      # Termination                                  
        prob=viterbi_scores[s,T]*transition[s].get('</s>',0)
        if prob>max_prob:
            max_prob=prob
            viterbi_scores['</s>',T+1]=prob
            backpointer['</s>',T+1]=s      
    tag_sequence=list()
    tag='</s>'
    tag_sequence.append(tag)
    for t in range(T+1,0,-1):
        tag=backpointer[tag,t]
        tag_sequence.append(tag)
    tag_sequence.reverse()
    tag_sequence=tag_sequence[1:len(tag_sequence)-1]
    return tag_sequence
tag_sequence=pos_tagger(sentence)

In [37]:
context.keys()

dict_keys(['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X'])

In [100]:
test_sentences=[[word[0] for word in sentence] for sentence in test_data]
test_tags=[[word[1] for word in sentence] for sentence in test_data]

In [101]:
predicted_tags=[pos_tagger(sentence) for sentence in test_sentences]

In [102]:
test_tags

[['PRON',
  'VERB',
  'NOUN',
  'ADV',
  'ADV',
  'PRON',
  'VERB',
  'DET',
  '.',
  'CONJ',
  'PRON',
  'VERB',
  'ADP',
  'PRON',
  'VERB',
  'VERB',
  'DET',
  'NOUN',
  '.',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'ADJ',
  'CONJ',
  'ADV',
  'VERB',
  'DET',
  'VERB',
  'ADP',
  'DET',
  'NOUN',
  '.',
  '.',
  'DET',
  'VERB',
  'DET',
  'ADJ',
  'ADJ',
  'NOUN',
  'ADP',
  'VERB',
  'PRON',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  '.'],
 ['ADJ',
  'NOUN',
  '.',
  'VERB',
  'PRON',
  'VERB',
  'ADP',
  'VERB',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  'ADV',
  'PRON',
  'VERB',
  '.'],
 ['DET',
  'NOUN',
  'ADP',
  'DET',
  'PRON',
  'VERB',
  'PRON',
  'ADP',
  'DET',
  'NOUN',
  'ADV',
  'VERB',
  'CONJ',
  'VERB',
  'CONJ',
  'ADJ',
  'CONJ',
  'DET',
  'ADJ',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'ADP',
  'ADJ',
  'NOUN',
  '.'],
 ['PRON',
  'VERB',
  '.',
  'PRON',
  'VERB',
  'ADV',
  'VERB',
  'PRT',
  'VERB',
  '.',
  'DET',
  'AD

In [107]:
def find_accuracy(predicted_tags, test_tags):
    num_sentences=len(predicted_tags)
    scores=list()
    for i in range(num_sentences):
        count=0
        predicted=predicted_tags[i]
        test=test_tags[i]
        len_sentence=len(test)
        for j in range(len_sentence):
            if(predicted[j]==test[j]):
                count+=1
            #else:
                #print(predicted[j],test[j])
        if(count/len_sentence<0.5):
            print(test_sentences[i])
            print(predicted)
            print(test)
        scores.append(count/len_sentence)
    #return np.mean(np.asarray(scores))
    return scores

In [108]:
find_accuracy(predicted_tags,test_tags)

['The', 'collective', 'by', 'which', 'I', 'address', 'you', 'in', 'the', 'title', 'above', 'is', 'neither', 'patronizing', 'nor', 'jocose', 'but', 'an', 'exact', 'industrial', 'term', 'in', 'use', 'among', 'professional', 'thieves', '.']
['DET', 'ADJ', 'ADP', 'DET', 'PRON', 'VERB', 'PRON', 'ADP', 'DET', 'NOUN', 'ADP', 'VERB', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET']
['DET', 'NOUN', 'ADP', 'DET', 'PRON', 'VERB', 'PRON', 'ADP', 'DET', 'NOUN', 'ADV', 'VERB', 'CONJ', 'VERB', 'CONJ', 'ADJ', 'CONJ', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', '.']
['It', 'is', ',', 'I', 'am', 'reliably', 'given', 'to', 'understand', ',', 'the', 'technical', 'argot', 'for', 'those', 'who', 'engage', 'in', 'your', 'particular', 'branch', 'of', 'the', 'boost', ';', ';']
['DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET', 'DET

[0.7551020408163265,
 0.95,
 0.4074074074074074,
 0.15384615384615385,
 0.34615384615384615,
 0.64,
 1.0,
 0.21739130434782608,
 0.5128205128205128,
 1.0,
 0.0,
 1.0,
 0.38095238095238093,
 0.45454545454545453,
 0.1875,
 0.3142857142857143,
 0.15789473684210525,
 0.25,
 0.125,
 0.38235294117647056,
 0.3225806451612903,
 0.5714285714285714,
 0.4782608695652174,
 0.9411764705882353,
 0.17073170731707318,
 0.42857142857142855,
 0.3783783783783784,
 0.5,
 0.325,
 0.59375,
 0.0,
 0.4482758620689655,
 0.0,
 0.42105263157894735,
 0.07142857142857142,
 0.13333333333333333,
 0.972972972972973,
 0.14285714285714285,
 0.42105263157894735,
 0.3333333333333333,
 0.08695652173913043,
 0.14285714285714285,
 0.0,
 0.0,
 0.9714285714285714,
 0.34782608695652173,
 0.15789473684210525,
 0.2727272727272727,
 0.14285714285714285,
 0.41379310344827586,
 0.6551724137931034,
 0.7727272727272727,
 0.875,
 0.8333333333333334,
 0.5833333333333334,
 0.3333333333333333,
 0.5625,
 1.0,
 0.18421052631578946,
 0.9473

In [109]:
emission['DET']

{'The': 0.05300459319504988,
 'an': 0.025723355080307774,
 'no': 0.01151955764898628,
 'any': 0.009391182235745005,
 'the': 0.45784207600713844,
 'which': 0.02584037915801176,
 'a': 0.1595257599251046,
 'this': 0.02895614522688043,
 'these': 0.008974283958924549,
 'both': 0.004702905122729002,
 'This': 0.008623211725812586,
 'our': 0.008323337526696117,
 'some': 0.010137210731107925,
 'its': 0.012989672625142622,
 'These': 0.0025233316754922326,
 'his': 0.046919341154442525,
 'His': 0.003847166554518592,
 'each': 0.005551329686082912,
 'what': 0.010473654954506889,
 'A': 0.008754863813229572,
 'every': 0.0031742781077206635,
 'their': 0.01867996840349902,
 'my': 0.008425733594687106,
 'Another': 0.0008118545390714139,
 'An': 0.0014408589567303472,
 'Some': 0.001499370995582341,
 'your': 0.00622421813288084,
 'What': 0.003379070243702642,
 'that': 0.014459787601298967,
 'another': 0.004183610777917557,
 'those': 0.005704923788069395,
 'Both': 0.0006363184225154326,
 'Every': 0.000416898